## Projet de MODS206 - Nowcasting and Dynamic Factor Models

### BASTIDE Simon, RISTORCELLI Ulysse, VUJASINOVIC Thomas

Analyse des bases de données de base, en se restreignant aux fréquences mensuelles :

In [2]:
#imports
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
pio.templates.default = "plotly_white"

In [15]:
#imports de capitalisations boursières des grandes entreprises du tourisme
def get_data(assets, start_date, end_date):
    """
    Récupère les données des actions sur Yahoo Finance
    """
    return yf.download(assets, start=start_date, end=end_date)['Adj Close']

assets = ['0H59.IL', 'ALVDM.PA', 'SW.PA', 'ELIOR.PA', 'AF.PA']
N = 4840 # Complétez avec le nombre de jours historiques

start_date = dt.datetime.now() - dt.timedelta(days=N)
end_date = dt.datetime.now()

data = get_data(assets, start_date, end_date)
data
monthly_data = data.resample('M').mean()
monthly_data


[*********************100%%**********************]  5 of 5 completed


,0H59.IL,AF.PA,ALVDM.PA,ELIOR.PA,SW.PA
Date,,,,,
2010-12-31,25.038467,68.944336,19.154295,NaN,24.036060
2011-01-31,25.038467,71.586155,19.968510,NaN,23.477961
2011-02-28,26.506670,63.725596,17.673535,NaN,23.480184
2011-03-31,25.664732,58.641388,17.225178,NaN,23.360513
2011-04-30,24.818356,58.330469,16.694480,NaN,24.565298
...,...,...,...,...,...
2023-11-30,30.999091,11.932545,112.863636,2.109091,69.859970
2023-12-31,34.373947,12.981158,120.447368,2.639368,70.274131
2024-01-31,35.514091,12.302000,124.954545,2.705000,74.967375


In [4]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

In [5]:
sight_seeing_demand_df=pd.read_csv('databases/famille_SUIVI-DEMANDE-TOURISTIQUE/valeurs_mensuelles.csv', sep=';')
sight_seeing_demand_df=sight_seeing_demand_df.drop(sight_seeing_demand_df[sight_seeing_demand_df['Libellé'] == 'Codes'].index)
sight_seeing_demand_df=sight_seeing_demand_df.drop('Période', axis=1)
sight_seeing_demand_df

overnight_stays_df=pd.read_csv('databases/famille_NUITEE/valeurs_mensuelles.csv', sep=';')
overnight_stays_df=overnight_stays_df.drop(overnight_stays_df[overnight_stays_df['Libellé'] == 'Codes'].index)
overnight_stays_df=overnight_stays_df.drop('Période',axis=1)

regions=[
    "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne",
    "Centre-Val de Loire", "Corse", "Grand Est", "Hauts-de-France",
    "Ile-de-France", "Normandie", "Nouvelle-Aquitaine", "Occitanie",
    "Pays de la Loire", "Provence Alpes Côte d’Azur"
]

overnight_stays_df_t = overnight_stays_df.transpose()
# Rename columns using the first row, cleansing of not useful columns
overnight_stays_df_t.columns = overnight_stays_df_t.iloc[0]
overnight_stays_df_t = overnight_stays_df_t.drop(overnight_stays_df_t.index[:3])

#overnight_stays_df_t = overnight_stays_df_t.iloc[:, list(range(3, 21)) + list(range(64, 80))]
occupation_columns = [col for col in overnight_stays_df_t.columns if "Taux d'occupation dans l'hôtellerie" in col]
overnight_columns = [col for col in overnight_stays_df_t.columns if "Nuitées dans l'hôtellerie - Total -" in col and "Série arrêtée" not in col]
arrival_columns= [col for col in overnight_stays_df_t.columns if "Arrivées dans l'hôtellerie - Total -" in col and "Série arrêtée" not in col]

selected_occupation_columns = [col for col in occupation_columns if any(region in col for region in regions)]
selected_overnight_columns = [col for col in overnight_columns if any(region in col for region in regions)]
selected_arrival_columns = [col for col in arrival_columns if any(region in col for region in regions)]

selected_columns = selected_arrival_columns+selected_occupation_columns+selected_overnight_columns

overnight_stays_df_t =overnight_stays_df_t[selected_columns]
overnight_stays_df_t=overnight_stays_df_t['2011-01':]

overnight_stays_df_t

Libellé,Arrivées dans l'hôtellerie - Total - Occitanie,Arrivées dans l'hôtellerie - Total - Pays de la Loire,Arrivées dans l'hôtellerie - Total - Bourgogne-Franche-Comté,Arrivées dans l'hôtellerie - Total - Corse-du-Sud,Arrivées dans l'hôtellerie - Total - Grand Est,Arrivées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes,Arrivées dans l'hôtellerie - Total - Normandie,Arrivées dans l'hôtellerie - Total - Corse,Arrivées dans l'hôtellerie - Total - Hauts-de-France,Arrivées dans l'hôtellerie - Total - Nouvelle-Aquitaine,...,Nuitées dans l'hôtellerie - Total - Normandie,Nuitées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes,Nuitées dans l'hôtellerie - Total - Grand Est,Nuitées dans l'hôtellerie - Total - Nouvelle-Aquitaine,Nuitées dans l'hôtellerie - Total - Occitanie,Nuitées dans l'hôtellerie - Total - Corse-du-Sud,Nuitées dans l'hôtellerie - Total - Bourgogne-Franche-Comté,Nuitées dans l'hôtellerie - Total - Haute-Corse,Nuitées dans l'hôtellerie - Total - Centre-Val de Loire,Nuitées dans l'hôtellerie - Total - Hauts-de-France
2011-01,418,233,257,9.75,439,902,223,17.0,328,404,...,344,1792,682,621,642,16.74,365,12.49,296,520
2011-02,468,248,295,11.59,460,988,275,21.0,352,461,...,416,2052,718,709,763,20.41,414,15.17,287,542
2011-03,534,286,352,15.85,564,1060,330,32.0,414,599,...,504,2149,902,949,852,29.76,489,24.64,363,651
2011-04,738,344,407,56.49,677,969,448,100.0,488,784,...,722,1682,1085,1286,1290,111.76,564,80.25,471,755
2011-05,849,352,438,115.04,713,942,450,194.0,493,807,...,726,1507,1148,1345,1538,221.77,614,153.34,546,775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,973,459,506,122.19,872,1303,529,227.0,562,1004,...,838,2053,1419,1654,1691,274.56,712,206.43,613,886
2023-10,801,393,406,68.27,770,1044,420,117.3,508,843,...,674,1650,1238,1369,1362,141.22,581,87.56,511,805
2023-11,524,301,293,15.91,623,837,296,31.0,406,584,...,470,1330,1005,913,838,26.35,427,23.84,369,662
2023-12,542,259,313,13.38,823,1070,264,25.9,430,596,...,421,1899,1357,891,834,20.76,427,19.56,353,670


In [6]:
change_rate_pd=pd.read_csv('databases/euro change rates.csv',sep=',')

change_rate_pd['Date'] = pd.to_datetime(change_rate_pd['Date'])

change_rate_pd=change_rate_pd[change_rate_pd['Date']>"2010-12-31"]
change_rate_pd.drop(columns=['Unnamed: 42'], inplace=True)

change_rate_pd = change_rate_pd.resample('M', on='Date').mean()

change_rate_pd

,USD,JPY,BGN,CYP,CZK,DKK,EEK,GBP,HUF,LTL,...,ILS,INR,KRW,MXN,MYR,NZD,PHP,SGD,THB,ZAR
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-31,1.335967,110.380952,1.9558,NaN,24.448810,7.451838,NaN,0.847117,275.326190,3.4528,...,4.790919,60.716086,1495.502857,16.192614,4.089490,1.743543,59.088952,1.719300,40.826571,9.265152
2011-02-28,1.364895,112.768000,1.9558,NaN,24.276550,7.455480,NaN,0.846355,271.152500,3.4528,...,4.993850,62.014225,1524.990500,16.472735,4.154105,1.792520,59.557600,1.742120,41.917500,9.812615
2011-03-31,1.399917,114.397391,1.9558,NaN,24.393000,7.457430,NaN,0.866533,270.890435,3.4528,...,4.986696,62.952635,1568.054348,16.806339,4.248304,1.887739,60.870391,1.775735,42.506391,9.686152
2011-04-30,1.444179,120.415263,1.9558,NaN,24.300895,7.457405,NaN,0.882913,265.287368,3.4528,...,4.957332,64.112789,1567.515789,16.921132,4.350226,1.833079,62.361211,1.802416,43.434474,9.719958
2011-05-31,1.434864,116.467273,1.9558,NaN,24.381318,7.456614,NaN,0.877877,266.958182,3.4528,...,4.974045,64.473455,1555.989545,16.717650,4.327218,1.802441,61.952773,1.776250,43.397955,9.846141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,1.080823,161.844091,1.9558,NaN,24.484773,7.458086,NaN,0.870454,379.194545,NaN,...,4.111768,90.010200,1412.571818,18.787691,5.066686,1.803295,60.284591,1.457527,38.327364,20.039686
2023-12-31,1.090305,157.212632,1.9558,NaN,24.477579,7.455637,NaN,0.861683,381.803158,NaN,...,4.005658,90.800937,1423.126316,18.766526,5.082553,1.755584,60.593000,1.453263,38.175579,20.337326
2024-01-31,1.090514,159.458182,1.9558,NaN,24.715818,7.457200,NaN,0.858731,382.041818,NaN,...,4.049314,90.634745,1447.184091,18.626832,5.110464,1.767264,61.089636,1.456873,38.400682,20.516095


In [7]:
cpi_index_df=pd.read_csv("databases/CPI deflated EER-41Euro.csv",sep=',')
pd.to_datetime(cpi_index_df['DATE'])
cpi_index_df=cpi_index_df[cpi_index_df['DATE']>"2010-12-31"]
cpi_index_df


,DATE,TIME PERIOD,CPI deflated EER-41/Euro (EXR.M.E03.EUR.ERC0.A)
216,2011-01-31,2011Jan,96.7008
217,2011-02-28,2011Feb,97.8516
218,2011-03-31,2011Mar,99.3829
219,2011-04-30,2011Apr,100.9849
220,2011-05-31,2011May,99.9160
...,...,...,...
369,2023-10-31,2023Oct,95.0032
370,2023-11-30,2023Nov,95.3326
371,2023-12-31,2023Dec,94.8389
372,2024-01-31,2024Jan,95.1396
